#Status Pipeline

1. Ingest status data from json files to bronze streaming table

2. Create streaming table for silver data 

3. Create materialized view for gold data joining status and orders data

4. Create materialized views for orders cancelled and orders delivered

## A. Ingest JSON -> Bronze

In [0]:
CREATE OR REFRESH STREAMING TABLE 1_bronze_db.status_bronze
  COMMENT "Ingest status data from JSON files"
  TBLPROPERTIES (
    "quality" = "bronze"
    --,"pipelines.reset.allowed" = false
  )
AS
SELECT
  *,
  current_timestamp() processing_time,
  _metadata.file_name AS source_file
FROM STREAM read_files(
    "${source}/status",
    format => "json" );

## B. Bronze => Silver

In [0]:
CREATE OR REFRESH STREAMING TABLE 2_silver_db.status_silver
  (
    CONSTRAINT valid_timestamp EXPECT (order_status_timestamp > "2021-12-25") ON VIOLATION DROP ROW,
    CONSTRAINT valid_status EXPECT (order_status IN ('on the way', 'cancelled', 'return cancelled', 'delivered', 'return processed', 'placed', 'preparing'))
  )
  COMMENT "Order with each status and timestamp"
  TBLPROPERTIES ( "quality" = "silver" )
AS
SELECT
  order_id,
  order_status,
  timestamp(status_timestamp) AS order_status_timestamp
FROM STREAM 1_bronze_db.status_bronze;

## C. Join two streaming tables

In [0]:
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.full_order_info_gold
  COMMENT "Joining the orders and order status silver tables to view full order info"
  TBLPROPERTIES ( "quality" = "gold" )
AS
SELECT
  orders.order_id,
  orders.order_timestamp,
  status.order_status,
  status.order_status_timestamp
FROM  2_silver_db.status_silver AS status
JOIN  2_silver_db.orders_silver AS orders
ON orders.order_id = status.order_id;

## D. Create Materialized Views for Cancelling and Delivered Orders

In [0]:
-- All cancelled orders MV
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.cancelled_orders_gold
  COMMENT "All cancelled orders"
  TBLPROPERTIES ( "quality" = "gold" )
AS
SELECT
  order_id,
  order_timestamp,
  order_status,
  order_status_timestamp,
  datediff(
    DAY,
    order_timestamp,
    order_status_timestamp
  ) AS days_to_cancel
FROM 3_gold_db.full_order_info_gold
WHERE order_status = 'cancelled';

-- All delivered orders MV
CREATE OR REFRESH MATERIALIZED VIEW 3_gold_db.delivered_orders_gold
  COMMENT "All delivered orders"
  TBLPROPERTIES ( "quality" = "gold" )
AS
SELECT
  order_id,
  order_timestamp,
  order_status,
  order_status_timestamp,
  datediff(
    DAY,
    order_timestamp,
    order_status_timestamp
  ) AS days_to_deliver
FROM 3_gold_db.full_order_info_gold
WHERE order_status = 'delivered';
